Scenario: Let's say we're working in a marketing department at Ozo Coffee (Links to an external site.), a local Boulder coffee shop that roasts some killer beans. They're looking to expand their roastery, and they're looking at new areas in which they can begin to market their online coffee ordering store to. They'd like to start with some billboards in specific cities to get the word out.

Ozo is a relatively new company. We don't know much about our target audience outside of Boulder. Luckily, Mintel, one of the leading Market research companies, just did a report on who drinks premium coffee.

Ozo is unique because we actually roast organic coffee, a pretty rare quality for coffee. Your boss tells you that your ads are going to use this key differentiator as the unique selling proposition for Ozo.

Let's start by getting a list of possible cities. Your boss is indifferent, and says that we need to start with the "largest cities." After some reading, you think that metro areas make more sense than specific cities, because metro areas tend to have more data on them, and people in the suburbs of cities also like coffee. You go to the Census website, and look for a list of MSAs. You find this page, and poke around (Links to an external site.), download an excel file with all of the MSAs. You notice that each MSA has a CBSA. CBSAs are codes that corresponds directly to a specific geofenced area in the US.  Fetch the data file directly from https://s3.amazonaws.com/vargo.aprd6342/data/msas.csv (Links to an external site.)

There are three columns in the data, the CBSA code, the name of the area, and the 2017 estimated population for that area. Get this CSV loaded into a dataframe.

Now, let's use CensusReporter, a very cool census data organization and visualization tool, and take a look at one of the metro areas, New York-Newark-Jersey City, NY-NJ-PA Metro Area (Links to an external site.). Inspect the charts. Think about what data would you extract from these charts to get your answers.

Let's take a look at the url: https://censusreporter.org/profiles/31000US35620-new-york-newark-jersey-city-ny-nj-pa-metro-area/ (Links to an external site.)

From the url above, you can see that each metro area has a code. It's intentional that what follows the "US" in the URL is the CBSA. So if we look at this part of the URL: 31000US35620, the CBSA is 35620.

Now it's time to get the data we need for the CBSA codes we have. Let's go back to our example MSA. Explore the page. Find the chart(s) that have the data you want (that match the three parameters from earlier).

In [0]:
import pandas as pd
import numpy as np
import requests
import json

In [0]:
DATA_URL = 'https://s3.amazonaws.com/vargo.aprd6342/data/msas.csv'
df = pd.read_csv(DATA_URL)

Change CBSA column to include US Code


In [0]:
df['CBSA2'] = '31000US'+ df['CBSA'].astype(str)

Set the new code to the index


In [0]:
df = df.set_index('CBSA2')

Drop the old CBSA codes (now that it has CBSA + USA codes as index)



In [0]:
df.drop('CBSA',axis=1,inplace=True)

Create an array of tables needed for census data

In [0]:
tableids = ['B01001', 'B03002', 'B19001']

In [0]:
tableidstring = ",".join(tableids)

create a URL for getting data crom censusreporter with table that we want to compare

In [0]:
requesturl = 'https://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=310|01000US' % tableidstring

In [0]:
requesturl

'https://api.censusreporter.org/1.0/data/show/latest?table_ids=B01001,B03002,B19001&geo_ids=310|01000US'

In [0]:
loadedjson = requests.get(requesturl)

In [0]:
parsedjson = json.loads(loadedjson.text)

create another data frame with the income, age, and hispanic population information

In [0]:
newdf = pd.DataFrame({ '50 - 59k': { id_: data['B19001']['estimate']['B19001011'] for id_, data in parsedjson['data'].items() }})
newdf['60 - 75k'] = pd.DataFrame({ '60 - 75k': { id_: data['B19001']['estimate']['B19001012'] for id_, data in parsedjson['data'].items() }})
newdf['18 and 19 years'] = pd.DataFrame({ '18 and 19 years': { id_: data['B01001']['estimate']['B01001007'] for id_, data in parsedjson['data'].items() }})
newdf['20 years'] = pd.DataFrame({ '20 years': { id_: data['B01001']['estimate']['B01001008'] for id_, data in parsedjson['data'].items() }})
newdf['21 years'] = pd.DataFrame({ '21 years': { id_: data['B01001']['estimate']['B01001009'] for id_, data in parsedjson['data'].items() }})
newdf['22 to 24 years'] = pd.DataFrame({ '22 to 24 years': { id_: data['B01001']['estimate']['B01001010'] for id_, data in parsedjson['data'].items() }})
newdf['25 to 29 years'] = pd.DataFrame({ '25 to 29 years': { id_: data['B01001']['estimate']['B01001011'] for id_, data in parsedjson['data'].items() }})
newdf['30 to 34 years'] = pd.DataFrame({ '30 to 34 years': { id_: data['B01001']['estimate']['B01001012'] for id_, data in parsedjson['data'].items() }})
newdf['35 to 39 years'] = pd.DataFrame({ '35 to 39 years': { id_: data['B01001']['estimate']['B01001013'] for id_, data in parsedjson['data'].items() }})
newdf['40 to 44 years'] = pd.DataFrame({ '40 to 44 years': { id_: data['B01001']['estimate']['B01001014'] for id_, data in parsedjson['data'].items() }})
newdf['Hispanic'] = pd.DataFrame({ 'Hispanic or Latino': { id_: data['B03002']['estimate']['B03002012'] for id_, data in parsedjson['data'].items() }})
newdf['total 50-75k'] = newdf['50 - 59k'] + newdf['60 - 75k']
newdf['Total 18 - 34'] = newdf['18 and 19 years'] + newdf['20 years'] + newdf['22 to 24 years'] + newdf['25 to 29 years'] + newdf['30 to 34 years'] + newdf['21 years']
newdf['Total 35 - 44'] = newdf['35 to 39 years'] + newdf['40 to 44 years'] 

merge the two dataframes on the index

In [0]:
df_index = pd.merge(df, newdf, right_index=True, left_index=True)

In [0]:
df_index

,NAME,POPESTIMATE2017,50 - 59k,60 - 75k,18 and 19 years,20 years,21 years,22 to 24 years,25 to 29 years,30 to 34 years,35 to 39 years,40 to 44 years,Hispanic,total 50-75k,Total 18 - 34,Total 35 - 44,income percent,age percent,Hispanic percent,Average Score
31000US10180,"Abilene, TX",170219,5409.0,6161.0,2955.0,1635.0,1496.0,5711.0,7218.0,6385.0,5757.0,4405.0,38892.0,11570.0,25400.0,10162.0,0.011010,0.010617,0.006512,0.009380
31000US10420,"Akron, OH",703505,23051.0,29395.0,10724.0,5803.0,4554.0,16178.0,23309.0,20623.0,19728.0,20218.0,13318.0,52446.0,81191.0,39946.0,0.049906,0.033939,0.002230,0.028692
31000US10500,"Albany, GA",151434,4471.0,5058.0,2270.0,1614.0,832.0,2939.0,5062.0,4454.0,4262.0,4615.0,4057.0,9529.0,17171.0,8877.0,0.009068,0.007178,0.000679,0.005641
31000US10540,"Albany, OR",125047,3859.0,5414.0,1279.0,977.0,606.0,2145.0,4010.0,3672.0,3614.0,3604.0,10483.0,9273.0,12689.0,7218.0,0.008824,0.005304,0.001755,0.005294
31000US10580,"Albany-Schenectady-Troy, NY",886188,26429.0,35665.0,15282.0,7901.0,7458.0,20534.0,28615.0,27678.0,25725.0,25795.0,42999.0,62094.0,107468.0,51520.0,0.059087,0.044923,0.007199,0.037070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31000US49420,"Yakima, WA",250193,7386.0,9514.0,3721.0,1979.0,1667.0,5495.0,8449.0,7871.0,7374.0,7469.0,120080.0,16900.0,29182.0,14843.0,0.016082,0.012198,0.020105,0.016128
31000US49620,"York-Hanover, PA",446078,15360.0,20503.0,5575.0,2900.0,2641.0,8006.0,13415.0,13316.0,13588.0,13347.0,30643.0,35863.0,45853.0,26935.0,0.034126,0.019167,0.005130,0.019475
31000US49660,"Youngstown-Warren-Boardman, OH-PA",541926,18929.0,23845.0,6912.0,3860.0,3478.0,10593.0,16119.0,14771.0,14631.0,15957.0,17895.0,42774.0,55733.0,30588.0,0.040703,0.023297,0.002996,0.022332
31000US49700,"Yuba City, CA",173679,4694.0,6480.0,2098.0,1257.0,1506.0,3628.0,6703.0,5983.0,5343.0,5184.0,49389.0,11174.0,21175.0,10527.0,0.010633,0.008851,0.008269,0.009251


Create columns to answer these questions: 
What city has the highest count of the audience you chose for parameter?
What city has the highest count of the audience you chose for parameter?
What city has the highest count of the audience you chose for parameter?
What city has the highest mean percentage across all three categories?

In [0]:
df_index['income percent'] = df_index['total 50-75k'] / 1050888
df_index['age percent'] = df_index['Total 18 - 34'] / 2392291
df_index['Hispanic percent'] = df_index['Hispanic'] / 5972751
df_index['Average Score'] = (df_index['income percent'] + df_index['age percent'] + df_index['Hispanic percent']) / 3

In [0]:
dfincome = df_index.sort_values('total 50-75k', ascending=False)
dfincome.head()

,NAME,POPESTIMATE2017,50 - 59k,60 - 75k,18 and 19 years,20 years,21 years,22 to 24 years,25 to 29 years,30 to 34 years,35 to 39 years,40 to 44 years,Hispanic,total 50-75k,Total 18 - 34,Total 35 - 44
31000US35620,"New York-Newark-Jersey City, NY-NJ-PA",20320876,444453.0,606435.0,248434.0,133732.0,132580.0,411899.0,750663.0,714983.0,660955.0,643736.0,4863602.0,1050888.0,2392291.0,1304691.0
31000US31080,"Los Angeles-Long Beach-Anaheim, CA",13353907,300860.0,399821.0,180803.0,100836.0,101477.0,294903.0,544334.0,497768.0,453173.0,451470.0,5972751.0,700681.0,1720121.0,904643.0
31000US16980,"Chicago-Naperville-Elgin, IL-IN-WI",9533040,250089.0,334288.0,126882.0,64973.0,63741.0,197802.0,345310.0,340707.0,317503.0,314667.0,2081524.0,584377.0,1139415.0,632170.0
31000US19100,"Dallas-Fort Worth-Arlington, TX",7399662,199823.0,253973.0,93659.0,46860.0,47524.0,143044.0,257776.0,258386.0,248173.0,248445.0,2018504.0,453796.0,847249.0,496618.0
31000US26420,"Houston-The Woodlands-Sugar Land, TX",6892427,172121.0,218920.0,87893.0,46904.0,44087.0,138637.0,250268.0,255333.0,237336.0,234011.0,2432651.0,391041.0,823122.0,471347.0


In [0]:
dfage = df_index.sort_values('Total 18 - 34', ascending=False)
dfage.head()

,NAME,POPESTIMATE2017,50 - 59k,60 - 75k,18 and 19 years,20 years,21 years,22 to 24 years,25 to 29 years,30 to 34 years,35 to 39 years,40 to 44 years,Hispanic,total 50-75k,Total 18 - 34,Total 35 - 44
31000US35620,"New York-Newark-Jersey City, NY-NJ-PA",20320876,444453.0,606435.0,248434.0,133732.0,132580.0,411899.0,750663.0,714983.0,660955.0,643736.0,4863602.0,1050888.0,2392291.0,1304691.0
31000US31080,"Los Angeles-Long Beach-Anaheim, CA",13353907,300860.0,399821.0,180803.0,100836.0,101477.0,294903.0,544334.0,497768.0,453173.0,451470.0,5972751.0,700681.0,1720121.0,904643.0
31000US16980,"Chicago-Naperville-Elgin, IL-IN-WI",9533040,250089.0,334288.0,126882.0,64973.0,63741.0,197802.0,345310.0,340707.0,317503.0,314667.0,2081524.0,584377.0,1139415.0,632170.0
31000US19100,"Dallas-Fort Worth-Arlington, TX",7399662,199823.0,253973.0,93659.0,46860.0,47524.0,143044.0,257776.0,258386.0,248173.0,248445.0,2018504.0,453796.0,847249.0,496618.0
31000US26420,"Houston-The Woodlands-Sugar Land, TX",6892427,172121.0,218920.0,87893.0,46904.0,44087.0,138637.0,250268.0,255333.0,237336.0,234011.0,2432651.0,391041.0,823122.0,471347.0


In [0]:
dfhispanic = df_index.sort_values('Hispanic', ascending=False)
dfhispanic.head()

,NAME,POPESTIMATE2017,50 - 59k,60 - 75k,18 and 19 years,20 years,21 years,22 to 24 years,25 to 29 years,30 to 34 years,35 to 39 years,40 to 44 years,Hispanic,total 50-75k,Total 18 - 34,Total 35 - 44
31000US31080,"Los Angeles-Long Beach-Anaheim, CA",13353907,300860.0,399821.0,180803.0,100836.0,101477.0,294903.0,544334.0,497768.0,453173.0,451470.0,5972751.0,700681.0,1720121.0,904643.0
31000US35620,"New York-Newark-Jersey City, NY-NJ-PA",20320876,444453.0,606435.0,248434.0,133732.0,132580.0,411899.0,750663.0,714983.0,660955.0,643736.0,4863602.0,1050888.0,2392291.0,1304691.0
31000US33100,"Miami-Fort Lauderdale-West Palm Beach, FL",6158824,158933.0,196989.0,69994.0,38315.0,37304.0,115197.0,207123.0,199215.0,195184.0,201129.0,2663235.0,355922.0,667148.0,396313.0
31000US26420,"Houston-The Woodlands-Sugar Land, TX",6892427,172121.0,218920.0,87893.0,46904.0,44087.0,138637.0,250268.0,255333.0,237336.0,234011.0,2432651.0,391041.0,823122.0,471347.0
31000US40140,"Riverside-San Bernardino-Ontario, CA",4580670,104313.0,139518.0,68012.0,37410.0,36922.0,102920.0,168330.0,154960.0,142360.0,141202.0,2239029.0,243831.0,568554.0,283562.0


In [0]:
dfscore = df_index.sort_values('Average Score', ascending=False)
dfscore.head()

,NAME,POPESTIMATE2017,50 - 59k,60 - 75k,18 and 19 years,20 years,21 years,22 to 24 years,25 to 29 years,30 to 34 years,35 to 39 years,40 to 44 years,Hispanic,total 50-75k,Total 18 - 34,Total 35 - 44,income percent,age percent,Hispanic percent,Average Score
31000US35620,"New York-Newark-Jersey City, NY-NJ-PA",20320876,444453.0,606435.0,248434.0,133732.0,132580.0,411899.0,750663.0,714983.0,660955.0,643736.0,4863602.0,1050888.0,2392291.0,1304691.0,1.000000,1.000000,0.814298,0.938099
31000US31080,"Los Angeles-Long Beach-Anaheim, CA",13353907,300860.0,399821.0,180803.0,100836.0,101477.0,294903.0,544334.0,497768.0,453173.0,451470.0,5972751.0,700681.0,1720121.0,904643.0,0.666751,0.719027,1.000000,0.795259
31000US16980,"Chicago-Naperville-Elgin, IL-IN-WI",9533040,250089.0,334288.0,126882.0,64973.0,63741.0,197802.0,345310.0,340707.0,317503.0,314667.0,2081524.0,584377.0,1139415.0,632170.0,0.556079,0.476286,0.348503,0.460290
31000US19100,"Dallas-Fort Worth-Arlington, TX",7399662,199823.0,253973.0,93659.0,46860.0,47524.0,143044.0,257776.0,258386.0,248173.0,248445.0,2018504.0,453796.0,847249.0,496618.0,0.431821,0.354158,0.337952,0.374644
31000US26420,"Houston-The Woodlands-Sugar Land, TX",6892427,172121.0,218920.0,87893.0,46904.0,44087.0,138637.0,250268.0,255333.0,237336.0,234011.0,2432651.0,391041.0,823122.0,471347.0,0.372105,0.344073,0.407292,0.374490


Answers to questions: 
1. The highest count for income 50 - 75k is the New York Metro Area
2. The highest count for age 18-34 is the New York Metro Area
3. The highest count for Hispanic population is the Los Angeles Metro Area
4. The highest percentage across all three is the New York Metro Area